##### Expand View


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

In [1]:
# Exploring MNIST Gans

import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST  # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/mnist_01')

import matplotlib.pyplot as plt
# import wandb

from markipy.nn.gans.generator import get_gen_loss, Generator
from markipy.nn.gans.discriminator import get_disc_loss, Discriminator
from markipy.nn.commons import show_tensor_images, make_noise
from markipy.basic import date


torch.manual_seed(0)  # Set for testing purposes, please do not change!


PROJECT="Playground"
VERSIONE="0.1"
RUN="0"


LOG_PATH=f'runs/{PROJECT}_{VERSIONE}.{date()}#{RUN}'

# Tensorboard
writer = SummaryWriter(LOG_PATH)


print(f"Tensorboard Logs path: {LOG_PATH}, localhost:6006")

Tensorboard Logs path: runs/Playground_0.1.2020.11.15#0, localhost:6006


## Torch PlayGround

In [2]:
batch_size = 32
ic = 1
iw = 28 
ih = 28


X = torch.randn(batch_size, ic,  iw, ih).cpu()
X.shape

torch.Size([32, 1, 28, 28])

In [38]:

from markipy.nn.commons import get_conv2d_block

n_sampled = X.shape[0]
oc = 5


cv1 = get_conv2d_block(ic, oc, ks=5, normalize=True, activation=nn.LeakyReLU(0.2))
m = nn.Upsample(scale_factor=1.2, mode='bilinear')
f = nn.Flatten(2)

x = X           ;print('Input  \t\t', x.shape)
x = m(x)        ;print('Upsampled  \t\t', x.shape)
x = cv1(x)      ;print('Conv2d  \t\t', x.shape)
x = f(x)        ;print('Flatten  \t\t', x.shape)


l_hidden = 10
Ln = [nn.Linear(1089, l_hidden) for n in range(oc)]
Xn = torch.zeros(n_sampled, oc, l_hidden)                   ;print(f'Pilars n{oc} \t\t', Xn.shape)


bidirectional = 2
ls_hidden = 10
h0 = torch.ones( bidirectional, n_sampled, l_hidden ) * 0.5
c0 = torch.ones( bidirectional, n_sampled, l_hidden ) * 0.5
lstm = nn.LSTM(l_hidden, ls_hidden ,1, dropout=0.88, bidirectional=True)   # Input (seq_len, batch, input_size)

ap2 = nn.AvgPool2d(2, stride=2)

cv2 = get_conv2d_block(1, 1, ks=3, p=2, normalize=False, activation=nn.LeakyReLU(0.2))

cv3 = get_conv2d_block(1, 1, ks=3, p=2, normalize=False, activation=nn.LeakyReLU(0.2))

cv4 = get_conv2d_block(1, 1, ks=3, p=2, normalize=False, activation=nn.LeakyReLU(0.2))

cv5 = get_conv2d_block(1, 1, ks=3, p=2, normalize=False, activation=nn.LeakyReLU(0.2))

cv6 = get_conv2d_block(1, 1, ks=3, p=2, normalize=False, activation=nn.LeakyReLU(0.2))

cv7 = get_conv2d_block(1, 1, ks=3, p=2, normalize=False, activation=nn.LeakyReLU(0.2))

cv8 = get_conv2d_block(1, 1, ks=3, p=2, normalize=False, activation=nn.LeakyReLU(0.2))

cv9 = get_conv2d_block(1, 1, ks=3, p=2, normalize=False, activation=nn.LeakyReLU(0.2))

cv10 = get_conv2d_block(1, 1, ks=3, p=2, normalize=False, activation=nn.LeakyReLU(0.2))

cv11 = get_conv2d_block(1, 1, ks=3, p=2, normalize=False, activation=nn.LeakyReLU(0.2))

for xi in range( oc ):

    Xn[: , xi , :] = Ln[xi](x[:,:1].view(n_sampled,1089))       ;print(f'Linear {xi + 1} \t\t', Xn[: , xi , :].shape)

x , (hn, cn) = lstm (Xn.permute(1,0,2), (h0,c0))           ;print(f'Lstm \t\t\t {x.shape} | Plus h:{hn.shape} c:{c0.shape}',)

x = x.permute(1,0,2)                               ;print(f'Permeute \t\t {x.shape}',)

# x = ap2(x)                                    ;print(f'AvgPool2d \t\t {x.shape}',)

x = x.flatten(0)                              ;print(f'Flatten \t\t\t {x.shape}',)

x = x.view( n_sampled , 1,  10, 10)        ;print(f'View \t\t\t {x.shape}',)

x = cv2(x)                             ;print('Conv2d  \t\t', x.shape)

x = cv3(x)                        ;print('Conv2d  \t\t', x.shape)

x = cv4(x)                        ;print('Conv2d  \t\t', x.shape)

x = cv5(x)                        ;print('Conv2d  \t\t', x.shape)

x = cv6(x)                        ;print('Conv2d  \t\t', x.shape)

x = cv7(x)                        ;print('Conv2d  \t\t', x.shape)

x = cv8(x)                        ;print('Conv2d  \t\t', x.shape)

x = cv9(x)                        ;print('Conv2d  \t\t', x.shape)

x = cv10(x)                        ;print('Conv2d  \t\t', x.shape)

Input  		 torch.Size([32, 1, 28, 28])
Upsampled  		 torch.Size([32, 1, 33, 33])
Conv2d  		 torch.Size([32, 5, 33, 33])
Flatten  		 torch.Size([32, 5, 1089])
Pilars n5 		 torch.Size([32, 5, 10])
Linear 1 		 torch.Size([32, 10])
Linear 2 		 torch.Size([32, 10])
Linear 3 		 torch.Size([32, 10])
Linear 4 		 torch.Size([32, 10])
Linear 5 		 torch.Size([32, 10])
Lstm 			 torch.Size([5, 32, 20]) | Plus h:torch.Size([2, 32, 10]) c:torch.Size([2, 32, 10])
Permeute 		 torch.Size([32, 5, 20])
Flatten 			 torch.Size([3200])
View 			 torch.Size([32, 1, 10, 10])
Conv2d  		 torch.Size([32, 1, 12, 12])
Conv2d  		 torch.Size([32, 1, 14, 14])
Conv2d  		 torch.Size([32, 1, 16, 16])
Conv2d  		 torch.Size([32, 1, 18, 18])
Conv2d  		 torch.Size([32, 1, 20, 20])
Conv2d  		 torch.Size([32, 1, 22, 22])
Conv2d  		 torch.Size([32, 1, 24, 24])
Conv2d  		 torch.Size([32, 1, 26, 26])
Conv2d  		 torch.Size([32, 1, 28, 28])


In [39]:
print(x.shape)

device = 'cpu'
n_sample = 1
noise_c = 1
noise_w = noise_b = 28
noise = make_noise(n_sample, (noise_c, noise_w, noise_b), device=device)



torch.Size([32, 1, 28, 28])


In [42]:
xl , (hn, cn) = lstm (Xn.permute(1,0,2), (h0,c0))

print(xl.shape, hn.shape, cn.shape, h0.shape, c0.shape)

torch.Size([5, 32, 20]) torch.Size([2, 32, 10]) torch.Size([2, 32, 10]) torch.Size([2, 32, 10]) torch.Size([2, 32, 10])


In [ ]:
avg_p = nn.AvgPool2d(2, stride=2)

In [119]:
avg_p(xl).shape

torch.Size([32, 2, 400])

In [139]:
x.shape

torch.Size([32, 2, 400])

In [149]:
x.view( 32, 2,  20, 20).shape

torch.Size([32, 2, 20, 20])

In [141]:
800 * 32

25600

In [146]:
28 * 28

784